# CMIP6 Future Predictions IPCC SSP1-1.9

## General python imports

In [1]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import geopandas as gpd
import pandas as pd
import seaborn as sns

#niceties
from rich import print

# Needed
from ipywidgets import IntProgress
from IPython.display import display
from scipy.stats import qmc

In [2]:
# general eWaterCycle
import ewatercycle
import ewatercycle.models
import ewatercycle.forcing

## Defining the experiment and file paths

In [3]:
# Defining the data period
start_historic = '1986-11-01'
end_historic = '2014-10-31'

future_data_start = '2026-11-01'
future_data_end = '2099-10-31' 

start_hydro_year = 1987
end_hydro_year = 2014

start_future_hydro_year = 2027
end_future_hydro_year = 2099

In [4]:
# defining path for catchment shape file
km_shp = Path.home() / "BEP-Zoe/book/thesis_projects/BSc/2025_Q4_ZoeLucius_CEG" /  "km_ERA5" / "km.shp"

# defining destination path for CMIP data
CMIP_folder = Path.home() / "CMIP"
CMIP_folder.mkdir(exist_ok=True)

# model HBV destination path
HBV_model_path = Path.home() / "HBV_model" / "CMIP"
HBV_model_path.mkdir(exist_ok=True)

## Importing the observed data

In [5]:
# The HBV model returns values in mm/day so in order to compare this to the observed data it must b converted to m^3/s
catchment_area_km2 = 334882.8

def mmday_to_m3s(Q_sim_mmday, area_km2):
    return (Q_sim_mmday * area_km2) / 86.4

In [6]:
# Loading in discharge data from Katima Mulilo
katima = (pd.read_csv('Katima_Mulilo/1291100_Q_Day.txt', skiprows=35, delimiter=';', header=1)
          .drop(columns=['hh:mm'])
          .rename(columns={"YYYY-MM-DD": "Date", " Value": "Discharge [m^3/s]"}))

# Creating a dataframe with accessible dates and no invalid data 
katima['Date'] = pd.to_datetime(katima['Date'])
katima['Discharge [m^3/s]'] = katima['Discharge [m^3/s]'].replace(-999.000, np.nan)

# Reducing the dataframe to the calibration period
km = katima[(katima['Date'] >= start_historic) & (katima['Date'] <= end_historic)]

# Creating a dataframe of the flow data 
flow = pd.DataFrame(data=km['Discharge [m^3/s]'].values, index=km['Date'], columns=['Q'])

## Importing the critical inflow per year

In [7]:
Q_crit = (pd.read_csv('Katima_Mulilo/critical_inflow.csv', usecols=[0, 3, 4, 6], delimiter=';', decimal='.')
          .rename(columns={'Year': 'hydro_year', 'Annual Power demand Zambia [MW]': 'power_demand_zambia [MW]', 'Annual Power demand Zimbabwe [MW]': 'power_demand_zimbabwe [MW]', 
                           'Required inflow at Katima Mulilo [m^3/s]': 'Q_crit'}))
Q_crit = Q_crit[(Q_crit['hydro_year'] >= start_future_hydro_year) & (Q_crit['hydro_year'] <= end_future_hydro_year)]

## Importing the drought analyser function

In [8]:
from Drought_Function import analyse_annual_deficits_MW

## Generating the forcings

In [9]:
CMIP_dataset = {'project': 'CMIP6', 'activity': 'ScenarioMIP', 'exp': 'ssp119', 'mip': 'day', 'dataset': 'MRI-ESM2-0', 'ensemble': 'r1i1p1f1', 'institute': 'MRI', 'grid': 'gn'}

# CMIP_forcing = ewatercycle.forcing.sources["LumpedMakkinkForcing"].generate(
#     dataset=CMIP_dataset,
#     start_time=future_data_start+'T00:00:00Z',
#     end_time=future_data_end+'T00:00:00Z',
#     shape=km_shp,
#     directory=CMIP_folder / 'scen_119')
   
CMIP_119 = CMIP_folder / "scen_119" / "work" / "diagnostic" / "script" 
SSP119 = ewatercycle.forcing.sources["LumpedMakkinkForcing"].load(directory=CMIP_119)

## Running the HBV model (with CMIP6 data)

In [10]:
# Applying the parameters found in the KGE model fit calibration
params_KGE = [5.65179127, 0.27593564, 779.15056566, 2.45298864, 0.04682875, 1.3263829, 0.01356608, 0.0070803, 0.48990002]

# Define the initial state parameters (Si,  Su, Sf, Ss, Sp)
s_0 = np.array([0,  100,  0,  5,  0])

In [11]:
model = ewatercycle.models.HBV(forcing=SSP119)

In [12]:
config_file, _ = model.setup(parameters=params_KGE, initial_storage=s_0, cfg_dir = HBV_model_path)

In [13]:
model.initialize(config_file)

In [14]:
Q_m = []
time = []

while model.time < model.end_time:
    model.update()
    Q_m.append(model.get_value("Q")[0])
    time.append(pd.Timestamp(model.time_as_datetime))

In [15]:
model.finalize()

## Visualising the data

In [16]:
CMIP_119_output = pd.DataFrame(data=Q_m, index=time, columns=['Q'])
CMIP_119_output = mmday_to_m3s(CMIP_119_output, catchment_area_km2)

In [17]:
QCMIP_119_sim = CMIP_119_output[future_data_start : future_data_end]
QCMIP_119_obs = flow[start_historic : end_historic]

QCMIP_119_sim.index = QCMIP_119_sim.index.normalize()
QCMIP_119_obs.index = QCMIP_119_obs.index.normalize()

In [18]:
# plot
plt.figure(figsize=(13, 8))

QCMIP_119_obs['Q'].plot(label="Observed Discharge", color='red')
QCMIP_119_sim['Q'].plot(label="Simulated Discharge", color='blue')

plt.title('Simulated vs Observed Discharge')
plt.xlabel('Date')
plt.ylabel('Discharge (m³/s)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Drought analysis

In [20]:
sim_119_droughts = QCMIP_119_sim.reset_index()
sim_119_droughts.rename(columns={'index': 'Date'}, inplace=True)

sim_119_droughts['Date'] = pd.to_datetime(sim_119_droughts['Date'])
sim_119_droughts['Year'] = sim_119_droughts['Date'].dt.year
sim_119_droughts['Day'] = sim_119_droughts['Date'].dt.dayofyear
sim_119_droughts['hydro_year'] = np.where(sim_119_droughts['Day'] < 305, sim_119_droughts['Year'], sim_119_droughts['Year'] + 1)

results119 = analyse_annual_deficits_MW(sim_119_droughts, 'Q', 'hydro_year', 1466.820, 730, 760)

results119['severity'].value_counts()

severity
mild        24
high        16
critical     4
Name: count, dtype: int64